In [2]:
import numpy as np
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import os
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')

In [3]:
engine = create_engine('postgresql+psycopg2://{user}:{pw}@{service}:{port}/{db}'.format(
    user = 'postgres',
    pw = POSTGRES_PASSWORD,
    service = 'postgres',
    port = '5432',
    db = 'nba'
))

In [8]:
myquery = '''
SELECT 
    game_date AS date, 
    season AS nba_season, 
    ot AS OT,
    2*ot AS two_times_ot
FROM games 
WHERE season=2022 AND ot>=2
ORDER BY ot DESC
LIMIT 20
'''
pd.read_sql_query(myquery, con=engine)

,date,nba_season,ot,two_times_ot
0,2021-11-26,2022,3,6
1,2022-01-29,2022,3,6
2,2022-02-17,2022,2,4
3,2021-10-20,2022,2,4
4,2021-10-30,2022,2,4
5,2021-11-27,2022,2,4


From the player_game table, make a table with all players who scored more than 30 points in the game. Keep only the columns with the game and player id, points (pts), and three point shots (fg3) made and attempted (fg3a). Sort by points in descending order, and display jus the top 15

In [23]:
myquery = '''
SELECT g.game_date, p.player, pg.pts, pg.fg3, pg.fg3a
FROM player_game pg
INNER JOIN players p
    ON pg.player_id = p.player_id
INNER JOIN games g
    ON pg.game_id = g.game_id
WHERE pg.pts > 30
ORDER BY pg.pts DESC
LIMIT 15
'''
pd.read_sql_query(myquery, con=engine)

,game_date,player,pts,fg3,fg3a
0,2021-01-03,Stephen Curry,62,8,16
1,2020-01-20,Damian Lillard,61,11,20
2,2020-08-11,Damian Lillard,61,9,17
3,2020-01-20,Damian Lillard,61,11,20
4,2020-08-11,Damian Lillard,61,9,17
5,2019-11-30,James Harden,60,8,14
6,2021-01-06,Bradley Beal,60,7,10
7,2021-04-30,Jayson Tatum,60,5,7
8,2019-11-08,Damian Lillard,60,7,16
9,2019-11-08,Damian Lillard,60,7,16


In the team game table, who won each game?
That will require you to join the team game to itself!
Keep the game id, the team and opponent abbreviation, the team score, and the other teams's score renamed to "opponent score"

In [26]:
myquery = '''
SELECT t.game_id,
    t.team_abbrev,
    t.opponent_abbrev,
    t.team_score,
    s.team_score AS opponent_score,
    (t.team_score > s.team_score) AS win
FROM team_game t
INNER JOIN team_game s
    ON t.game_id = s.game_id 
        AND t.team_abbrev = s.opponent_abbrev
'''
pd.read_sql_query(myquery, con=engine)

,game_id,team_abbrev,opponent_abbrev,team_score,opponent_score,win
0,202202170BRK,WAS,BRK,117,103,True
1,202202170BRK,BRK,WAS,103,117,False
2,202202170CHO,MIA,CHO,111,107,True
3,202202170CHO,CHO,MIA,107,111,False
4,202202170LAC,HOU,LAC,111,142,False
...,...,...,...,...,...,...
6389,202002270GSW,GSW,LAL,86,116,False
6390,202002290PHO,GSW,PHO,115,99,True
6391,202003010GSW,GSW,WAS,110,124,False
6392,202003030DEN,GSW,DEN,116,100,True
